#### <u> Mark Dingli - 20703H <u>
# <u> ICS2203 – Statistical Natural Language Processing <u>
# <u> Building a Language Model – Part I <u>

#### This Jupyter notebook draws inspiration from the content presented in:

J. Martin, and D. Jurafsky, "Speech and Language Processing: An Introduction to Natural Language Processing, Computational Linguistics, and Speech Recognition, 3rd ed.," Stanford University, 2020. [Online]. Available: https://web.stanford.edu/~jurafsky/slp3/3.pdf. [Accessed: March 10, 2023].

## Imports

In [1]:
# Standard Library Imports
import os
import re
import time
import math
import xml.etree.ElementTree as ET
from datetime import datetime
from collections import Counter
import json

# Third-Party Library Imports
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split

# System Library Imports
import psutil

## Extracting and Pre-processing the Selected Corpus

#### The corpus used in this project is the (Baby) British National Corpus: http://ota.ox.ac.uk/desc/2553

In [2]:
# Set directory path for corpus
corpus_dir_path = 'corpus'

# Create an empty list for tokens
tokens = []

# Instantiate a WordNetLemmatizer object
lemmatizer = WordNetLemmatizer()

# Instantiate a RegexpTokenizer object for tokenizing text
tokenizer = RegexpTokenizer(r"\w+(?:[-']\w+)*|[^\w\s]")

# Traverse all files in the specified directory and subdirectories, and tokenize text in them
for root_dir, subdirs, files in os.walk(corpus_dir_path):
    for filename in files:
        # Select only XML files
        if filename.endswith('.xml'):
            # Parse XML file
            tree = ET.parse(os.path.join(root_dir, filename))
            root = tree.getroot()
            text = ''
            for element in root.iter():
                if element.text is not None:
                    text += element.text + ' '

            # Remove URLs from text
            text = re.sub(r'https?://\S+|www\.\S+', '', text)  

            # Convert text to lowercase
            text = text.lower()

            # Tokenize text using RegexpTokenizer object
            file_tokens = tokenizer.tokenize(text)

            # Remove punctuation, numbers, and lemmatize tokens
            file_tokens = [lemmatizer.lemmatize(token) for token in file_tokens 
                           if not token.isdigit() and 
                           (token.isalpha() or len(token) > 1)]

            # Add the tokens to the tokens list
            tokens.extend(file_tokens)

# Output list of the first 100 tokens
print()
print(tokens[:100])


['oxford', 'art', 'journal', 'sample', 'containing', 'about', 'word', 'from', 'a', 'periodical', 'domain', 'art', 'data', 'capture', 'and', 'transcription', 'oxford', 'university', 'press', 'bnc', 'xml', 'edition', 'december', 'token', 'w-units', 's-units', 'distributed', 'under', 'licence', 'by', 'oxford', 'university', 'computing', 'service', 'on', 'behalf', 'of', 'the', 'bnc', 'consortium', 'this', 'material', 'is', 'protected', 'by', 'international', 'copyright', 'law', 'and', 'may', 'not', 'be', 'copied', 'or', 'redistributed', 'in', 'any', 'way', 'consult', 'the', 'bnc', 'web', 'site', 'at', 'for', 'full', 'licencing', 'and', 'distribution', 'condition', 'a6u', 'artjnl', 'oxford', 'art', 'journal', 'oxford', 'university', 'press', 'oxford', 'w', 'ac', 'humanity', 'art', 'art', 'tag', 'usage', 'updated', 'for', 'bnc-xml', 'last', 'check', 'for', 'bnc', 'world', 'first', 'release', 'redo', 'tagusage', 'table', 'check']


### Printing the size of list that includes all the words after preprocessing

In [3]:
# Print the total size of the token list
print("Total size of corpus: "+str(len(tokens))+" words")

Total size of corpus: 4003540 words


## Splitting the corpus tokens into test set and train set

I chose to make the test set 20% of the total corpus but that can easily be changed by adjusting the second variable.

In [4]:
# Split the words into train and test sets
train_set, test_set = train_test_split(tokens, test_size=0.2)

# Define the vocabulary from the training set
vocab = set()
for sentence in train_set:
    for word in sentence.split():
        vocab.add(word)
vocab = list(vocab)
vocab.sort()

# Print the size of the train and test sets
print("Train set size:", len(train_set))
print("Test set size:", len(test_set))

Train set size: 3202832
Test set size: 800708


# <u> Building a Language Model – Part II <u>

## Vanilla Language Model

In [17]:
# Define a function to get RAM usage
def RAMusage():
    pid = os.getpid()
    py = psutil.Process(pid)
    memoryUse = py.memory_info()[0] / 2.**30
    return memoryUse

# Record start time for model export
vanilla_export_start = datetime.now()

# Define function to create n-grams from a list of tokens
def create_vanilla_ngrams(tokens, n):
    return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

#-------------------------------------------------------------------------------------------------------------------------------

# Create a vanilla unigram model by iterating over the train_set and counting the frequency of each word
vanilla_unigram_freq = {}
vanilla_total_unigrams = len(train_set)
for word in train_set:
    if word in vanilla_unigram_freq:
        vanilla_unigram_freq[word] += 1
    else:
        vanilla_unigram_freq[word] = 1

# Calculate vanilla unigram probabilities by dividing the frequency of each word by the total number of unigrams
vanilla_unigram_probs = {}
for word, count in vanilla_unigram_freq.items():
    vanilla_unigram_probs[word] = count / vanilla_total_unigrams
    
#-------------------------------------------------------------------------------------------------------------------------------

# Create a vanilla bigram model by creating bigrams from the train_set and counting the frequency of each bigram
vanilla_bigram_freq = {}
vanilla_total_bigrams = len(create_vanilla_ngrams(train_set, 2))
vanilla_bigram_tokens = create_vanilla_ngrams(train_set, 2)
for tokens in vanilla_bigram_tokens:
    if tokens in vanilla_bigram_freq:
        vanilla_bigram_freq[tokens] += 1
    else:
        vanilla_bigram_freq[tokens] = 1

# Calculate vanilla bigram probabilities by dividing the frequency of each bigram by the frequency of its first word
vanilla_bigram_probs = {}
for tokens, count in vanilla_bigram_freq.items():
    vanilla_bigram_probs[tokens] = count / vanilla_unigram_freq[tokens[0]]
    
#-------------------------------------------------------------------------------------------------------------------------------

# Create a vanilla trigram model by creating trigrams from the train_set and counting the frequency of each trigram
vanilla_trigram_freq = {}
vanilla_total_trigrams = len(create_vanilla_ngrams(train_set, 3))
vanilla_trigram_tokens = create_vanilla_ngrams(train_set, 3)
for tokens in vanilla_trigram_tokens:
    if tokens in vanilla_trigram_freq:
        vanilla_trigram_freq[tokens] += 1
    else:
        vanilla_trigram_freq[tokens] = 1

# Calculate vanilla trigram probabilities by dividing the frequency of each trigram by the frequency of its first two words
vanilla_trigram_probs = {}
for tokens, count in vanilla_trigram_freq.items():
    vanilla_trigram_probs[tokens] = count / vanilla_bigram_freq[(tokens[0], tokens[1])]
    
#-------------------------------------------------------------------------------------------------------------------------------

## Computation
# Convert keys of vanilla_bigram_probs and vanilla_trigram_probs to strings
vanilla_bigram_probs_str = {str(k): v for k, v in vanilla_bigram_probs.items()}
vanilla_trigram_probs_str = {str(k): v for k, v in vanilla_trigram_probs.items()}

# Combine vanilla unigram, bigram, and trigram probabilities into a single dictionary
vanilla_probs = {
    'unigrams': vanilla_unigram_probs,
    'bigrams': vanilla_bigram_probs_str,
    'trigrams': vanilla_trigram_probs_str
}

# Save vanilla probabilities in a JSON file
with open('vanilla_language_model.json', 'w') as f:
    json.dump(vanilla_probs, f)

# Record end time for Vanilla model export
vanilla_export_end = datetime.now()

# Calculate the time taken to export Vanilla model
vanilla_export_time = vanilla_export_end - vanilla_export_start

# Get the size of the JSON file in bytes
vanilla_json_file_size = os.path.getsize('vanilla_language_model.json')

# Convert the file size to a human-readable format
for unit in ['B', 'KB', 'MB', 'GB']:
    if vanilla_json_file_size < 1024.0:
        vanilla_json_file_size_str = f"{vanilla_json_file_size:.2f} {unit}"
        break
    vanilla_json_file_size /= 1024.0


# Print the time taken to export Vanilla model
print('\nVanilla Language Model Build & Export Time(HH:MM:SS:ms) - {}\n'.format(vanilla_export_time))

# Print the memory used by Vanilla model in GB
print("Vanilla Language Model Memory Use: {:.6f} GB\n".format(RAMusage()))

# Print the size of the JSON file
print("Vanilla Language Model JSON File Size:", vanilla_json_file_size_str)
print()
print("-" * 123)


Vanilla Language Model Build & Export Time(HH:MM:SS:ms) - 0:00:34.357551

Vanilla Language Model Memory Use: 3.068428 GB

Vanilla Language Model JSON File Size: 174.03 MB

---------------------------------------------------------------------------------------------------------------------------



### Test output for Vanilla Language Model

In [6]:
## Testing Vanilla Language Model by printing subsets of the model and probability dictionaries

# Print the frequency of the first 10 unigrams in the model
print("\nVanilla unigram model (subset):")
print({k: vanilla_unigram_freq[k] for k in list(vanilla_unigram_freq)[:10]})

# Print the probability of the first 10 unigrams in the model
print("\nVanilla unigram probabilities (subset):")
print({k: vanilla_unigram_probs[k] for k in list(vanilla_unigram_probs)[:10]})
print("-"*127)

# Print the frequency of the first 10 bigrams in the model
print("\nVanilla bigram model (subset):")
print({k: vanilla_bigram_freq[k] for k in list(vanilla_bigram_freq)[:10]})

# Print the probability of the first 10 bigrams in the model
print("\nVanilla bigram probabilities (subset):")
print({k: vanilla_bigram_probs[k] for k in list(vanilla_bigram_probs)[:10]})
print("-"*127)

# Print the frequency of the first 10 trigrams in the model
print("\nVanilla trigram model (subset):")
print({k: vanilla_trigram_freq[k] for k in list(vanilla_trigram_freq)[:10]})

# Print the probability of the first 10 trigrams in the model
print("\nVanilla trigram probabilities (subset):")
print({k: vanilla_trigram_probs[k] for k in list(vanilla_trigram_probs)[:10]})
print("-"*127)


Vanilla unigram model (subset):
{'ever-dependable': 1, 'said': 10438, 'himself': 1069, 'do': 16385, 'it': 52279, 'see': 4949, 'in': 55405, 'from': 11840, 'shit': 166, 'that': 41623}

Vanilla unigram probabilities (subset):
{'ever-dependable': 3.122236820413934e-07, 'said': 0.003258990793148064, 'himself': 0.0003337671161022495, 'do': 0.00511578503024823, 'it': 0.016322741873442004, 'see': 0.0015451950024228558, 'in': 0.0172987531035034, 'from': 0.0036967283953700976, 'shit': 5.18291312188713e-05, 'that': 0.012995686317608916}
-------------------------------------------------------------------------------------------------------------------------------

Vanilla bigram model (subset):
{('ever-dependable', 'said'): 1, ('said', 'himself'): 3, ('himself', 'do'): 6, ('do', 'it'): 276, ('it', 'see'): 82, ('see', 'in'): 91, ('in', 'in'): 962, ('in', 'from'): 198, ('from', 'shit'): 1, ('shit', 'that'): 5}

Vanilla bigram probabilities (subset):
{('ever-dependable', 'said'): 1.0, ('said', 'hims

## Laplace Language Model

In [18]:
# Record start time for model export
laplace_export_start = datetime.now()

# Define function to create n-grams from a list of tokens
def create_laplace_ngrams(tokens, n):
    return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

#-------------------------------------------------------------------------------------------------------------------------------

# Create a Laplace unigram model by iterating over the train_set and counting the frequency of each word
laplace_unigram_freq = {}
laplace_total_unigrams = len(train_set)
for word in train_set:
    if word in laplace_unigram_freq:
        laplace_unigram_freq[word] += 1
    else:
        laplace_unigram_freq[word] = 1

# Calculate Laplace unigram probabilities by adding 1 to the frequency of each word and dividing by the total number of unigrams plus the vocabulary size
laplace_unigram_probs = {}
for word, count in laplace_unigram_freq.items():
    laplace_unigram_probs[word] = (count + 1) / (laplace_total_unigrams + len(laplace_unigram_freq))
    
#-------------------------------------------------------------------------------------------------------------------------------
    
# Create a Laplace bigram model by creating bigrams from the train_set and counting the frequency of each bigram
laplace_bigram_freq = {}
laplace_total_bigrams = len(create_laplace_ngrams(train_set, 2))
laplace_bigram_tokens = create_laplace_ngrams(train_set, 2)
for tokens in laplace_bigram_tokens:
    if tokens in laplace_bigram_freq:
        laplace_bigram_freq[tokens] += 1
    else:
        laplace_bigram_freq[tokens] = 1

# Calculate Laplace bigram probabilities by adding 1 to the frequency of each bigram and dividing by the frequency of its first word plus the vocabulary size
laplace_bigram_probs = {}
for tokens, count in laplace_bigram_freq.items():
    laplace_bigram_probs[tokens] = (count + 1) / (laplace_unigram_freq[tokens[0]] + len(laplace_unigram_freq))
    
#-------------------------------------------------------------------------------------------------------------------------------
    
# Create a Laplace trigram model by creating trigrams from the train_set and counting the frequency of each trigram
laplace_trigram_freq = {}
laplace_total_trigrams = len(create_laplace_ngrams(train_set, 3))
laplace_trigram_tokens = create_laplace_ngrams(train_set, 3)
for tokens in laplace_trigram_tokens:
    if tokens in laplace_trigram_freq:
        laplace_trigram_freq[tokens] += 1
    else:
        laplace_trigram_freq[tokens] = 1

# Calculate Laplace trigram probabilities by adding 1 to the frequency of each trigram and dividing by the frequency of its first two words plus the vocabulary size
laplace_trigram_probs = {}
for tokens, count in laplace_trigram_freq.items():
    laplace_trigram_probs[tokens] = (count + 1) / (laplace_bigram_freq[(tokens[0], tokens[1])] + len(laplace_unigram_freq))
    
#-------------------------------------------------------------------------------------------------------------------------------

## Computation
# Convert keys of laplace_bigram_probs and laplace_trigram_probs to strings
laplace_bigram_probs_str = {str(k): v for k, v in laplace_bigram_probs.items()}
laplace_trigram_probs_str = {str(k): v for k, v in laplace_trigram_probs.items()}

# Combine Laplace unigram, bigram, and trigram probabilities into a single dictionary
laplace_probs = {
    'unigrams': laplace_unigram_probs,
    'bigrams': laplace_bigram_probs_str,
    'trigrams': laplace_trigram_probs_str
}

# Save Laplace probabilities in a JSON file
with open('laplace_language_model.json', 'w') as f:
    json.dump(laplace_probs, f)

# Record end time for Laplace model export
laplace_export_end = datetime.now()

# Calculate the time taken to export Laplace model
laplace_export_time = laplace_export_end - laplace_export_start

# Get the size of the JSON file in bytes
laplace_json_file_size = os.path.getsize('laplace_language_model.json')

# Convert the file size to a human-readable format
for unit in ['B', 'KB', 'MB', 'GB']:
    if laplace_json_file_size < 1024.0:
        laplace_json_file_size_str = f"{laplace_json_file_size:.2f} {unit}"
        break
    laplace_json_file_size /= 1024.0

# Print the time taken to export Laplace model
print('\nLaplace Language Model Build & Export Time(HH:MM:SS:ms) - {}\n'.format(laplace_export_time))

# Print the memory used by Laplace model in GB
print("Laplace Language Model Memory Use: {:.6f} GB\n".format(RAMusage()))

# Print the size of the JSON file
print("Laplace Language Model JSON File Size:", laplace_json_file_size_str)
print()
print("-" * 123)


Laplace Language Model Build & Export Time(HH:MM:SS:ms) - 0:00:42.646727

Laplace Language Model Memory Use: 3.304901 GB

Laplace Language Model JSON File Size: 207.33 MB

---------------------------------------------------------------------------------------------------------------------------



### Test output for Laplace  Language Model

In [8]:
## Testing Laplace Language Model by printing subsets of the model and probability dictionaries

# Print the frequency of the first 10 unigrams in the model
print("\nLaplace unigram model (subset):")
print({k: laplace_unigram_freq[k] for k in list(laplace_unigram_freq)[:10]})

# Print the probability of the first 10 unigrams in the model
print("\nLaplace unigram probabilities (subset):")
print({k: laplace_unigram_probs[k] for k in list(laplace_unigram_probs)[:10]})
print("-"*127)

# Print the frequency of the first 10 bigrams in the model
print("\nLaplace bigram model (subset):")
print({k: laplace_bigram_freq[k] for k in list(laplace_bigram_freq)[:10]})

# Print the probability of the first 10 bigrams in the model
print("\nLaplace bigram probabilities (subset):")
print({k: laplace_bigram_probs[k] for k in list(laplace_bigram_probs)[:10]})
print("-"*127)

# Print the frequency of the first 10 trigrams in the model
print("\nLaplace trigram model (subset):")
print({k: laplace_trigram_freq[k] for k in list(laplace_trigram_freq)[:10]})

# Print the probability of the first 10 trigrams in the model
print("\nLaplace trigram probabilities (subset):")
print({k: laplace_trigram_probs[k] for k in list(laplace_trigram_probs)[:10]}) 
print("-"*127)


Laplace unigram model (subset):
{'ever-dependable': 1, 'said': 10438, 'himself': 1069, 'do': 16385, 'it': 52279, 'see': 4949, 'in': 55405, 'from': 11840, 'shit': 166, 'that': 41623}

Laplace unigram probabilities (subset):
{'ever-dependable': 6.120140187931145e-07, 'said': 0.003194407171090661, 'himself': 0.00032742750005431624, 'do': 0.005014230855971987, 'it': 0.015998046451252013, 'see': 0.0015147346965129584, 'in': 0.01695462436262565, 'from': 0.0036234289982646343, 'shit': 5.110317056922506e-05, 'that': 0.0127372357591223}
-------------------------------------------------------------------------------------------------------------------------------

Laplace bigram model (subset):
{('ever-dependable', 'said'): 1, ('said', 'himself'): 3, ('himself', 'do'): 6, ('do', 'it'): 276, ('it', 'see'): 82, ('see', 'in'): 91, ('in', 'in'): 962, ('in', 'from'): 198, ('from', 'shit'): 1, ('shit', 'that'): 5}

Laplace bigram probabilities (subset):
{('ever-dependable', 'said'): 3.07370750599373e

## UNK Language model

In [21]:
# Record start time for model export
unk_export_start = datetime.now()

# Create a new list with <UNK> tokens for words with count less than or equal to 2
unk_train_set = []
word_freq = {}
for word in train_set:
    if word in word_freq:
        word_freq[word] += 1
    else:
        word_freq[word] = 1
    if word_freq[word] <= 2:
        unk_train_set.append('<UNK>')
    else:
        unk_train_set.append(word)

# Create UNK Laplace unigram model
unk_laplace_unigram_freq = {}
unk_laplace_total_unigrams = len(unk_train_set)
for word in unk_train_set:
    if word in unk_laplace_unigram_freq:
        unk_laplace_unigram_freq[word] += 1
    else:
        unk_laplace_unigram_freq[word] = 1

# Calculate UNK Laplace unigram probabilities
unk_laplace_unigram_probs = {}
for word, count in unk_laplace_unigram_freq.items():
    unk_laplace_unigram_probs[word] = (count + 1) / (unk_laplace_total_unigrams + len(unk_laplace_unigram_freq))

#-------------------------------------------------------------------------------------------------------------------------------

# Create UNK Laplace bigram model
unk_laplace_bigram_freq = {}
unk_laplace_total_bigrams = len(create_laplace_ngrams(unk_train_set, 2))
unk_laplace_bigram_tokens = create_laplace_ngrams(unk_train_set, 2)
for tokens in unk_laplace_bigram_tokens:
    if tokens in unk_laplace_bigram_freq:
        unk_laplace_bigram_freq[tokens] += 1
    else:
        unk_laplace_bigram_freq[tokens] = 1

# Calculate UNK Laplace bigram probabilities
unk_laplace_bigram_probs = {}
for tokens, count in unk_laplace_bigram_freq.items():
    unk_laplace_bigram_probs[tokens] = (count + 1) / (unk_laplace_unigram_freq[tokens[0]] + len(unk_laplace_unigram_freq))

#-------------------------------------------------------------------------------------------------------------------------------

# Create UNK Laplace trigram model
unk_laplace_trigram_freq = {}
unk_laplace_total_trigrams = len(create_laplace_ngrams(unk_train_set, 3))
unk_laplace_trigram_tokens = create_laplace_ngrams(unk_train_set, 3)
for tokens in unk_laplace_trigram_tokens:
    if tokens in unk_laplace_trigram_freq:
        unk_laplace_trigram_freq[tokens] += 1
    else:
        unk_laplace_trigram_freq[tokens] = 1

# Calculate UNK Laplace trigram probabilities
unk_laplace_trigram_probs = {}
for tokens, count in unk_laplace_trigram_freq.items():
    unk_laplace_trigram_probs[tokens] = (count + 1) / (unk_laplace_bigram_freq[(tokens[0], tokens[1])] + len(unk_laplace_unigram_freq))
    
#-------------------------------------------------------------------------------------------------------------------------------

## Computation
# Convert keys of unk_laplace_bigram_probs and unk_laplace_trigram_probs to strings
unk_laplace_bigram_probs_str = {str(k): v for k, v in unk_laplace_bigram_probs.items()}
unk_laplace_trigram_probs_str = {str(k): v for k, v in unk_laplace_trigram_probs.items()}

# Combine UNK Laplace unigram, bigram, and trigram probabilities into a single dictionary
unk_laplace_probs = {
    'unigrams': unk_laplace_unigram_probs,
    'bigrams': unk_laplace_bigram_probs_str,
    'trigrams': unk_laplace_trigram_probs_str
}

# Save UNK Laplace probabilities in a JSON file
with open('unk_language_model.json', 'w') as f:
    json.dump(unk_laplace_probs, f)

# Record end time for UNK Laplace model export
unk_laplace_export_end = datetime.now()

# Calculate the time taken to export UNK Laplace model
unk_laplace_export_time = unk_laplace_export_end - unk_export_start

# Get the size of the JSON file in bytes
unk_json_file_size = os.path.getsize('unk_language_model.json')

# Convert the file size to a human-readable format
for unit in ['B', 'KB', 'MB', 'GB']:
    if unk_json_file_size < 1024.0:
        unk_json_file_size_str = f"{unk_json_file_size:.2f} {unit}"
        break
    unk_json_file_size /= 1024.0

# Print the time taken to export UNK Laplace model
print('\nUNK Language Model Build & Export Time(HH:MM:SS:ms) - {}\n'.format(unk_laplace_export_time))

# Print the memory used by UNK Laplace model in GB
print("UNK Language Model Memory Use: {:.6f} GB\n".format(RAMusage()))

# Print the size of the JSON file
print("UNK Language Model JSON File Size:", unk_json_file_size_str)
print()
print("-" * 123)


UNK Language Model Build & Export Time(HH:MM:SS:ms) - 0:00:37.303823

UNK Language Model Memory Use: 2.875320 GB

UNK Language Model JSON File Size: 192.83 MB

---------------------------------------------------------------------------------------------------------------------------



### Test output for UNK Language Model

In [10]:
## Testing UNK Language Model by printing subsets of the model and probability dictionaries

# Print the frequency of the first 10 unigrams in the model
print("\nUNK Laplace unigram model (subset):")
print({k: unk_laplace_unigram_freq[k] for k in list(unk_laplace_unigram_freq)[:10]})

# Print the probability of the first 10 unigrams in the model
print("\nUNK Laplace unigram probabilities (subset):")
print({k: unk_laplace_unigram_probs[k] for k in list(unk_laplace_unigram_probs)[:10]})
print("-"*127)

# Print the frequency of the first 10 bigrams in the model
print("\nUNK Laplace bigram model (subset):")
print({k: unk_laplace_bigram_freq[k] for k in list(unk_laplace_bigram_freq)[:10]})

# Print the probability of the first 10 bigrams in the model
print("\nUNK Laplace bigram probabilities (subset):")
print({k: unk_laplace_bigram_probs[k] for k in list(unk_laplace_bigram_probs)[:10]})
print("-"*127)

# Print the frequency of the first 10 trigrams in the model
print("\nUNK Laplace trigram model (subset):")
print({k: unk_laplace_trigram_freq[k] for k in list(unk_laplace_trigram_freq)[:10]})

# Print the probability of the first 10 trigrams in the model
print("\nUNK Laplace trigram probabilities (subset):")
print({k: unk_laplace_trigram_probs[k] for k in list(unk_laplace_trigram_probs)[:10]})
print("-"*127)


UNK Laplace unigram model (subset):
{'<UNK>': 102880, 'a': 90373, 'you': 39565, 'in': 55403, 'the': 169348, 'it': 52277, 'to': 75963, 'of': 80912, 'is': 32556, 'that': 41621}

UNK Laplace unigram probabilities (subset):
{'<UNK>': 0.031833509393702546, 'a': 0.027963584898537862, 'you': 0.01224253878433564, 'in': 0.017143143578004644, 'the': 0.05240008341981642, 'it': 0.016175894519726498, 'to': 0.023504832841663867, 'of': 0.02503615580692892, 'is': 0.010073809209968544, 'that': 0.012878707710701561}
-------------------------------------------------------------------------------------------------------------------------------

UNK Laplace bigram model (subset):
{('<UNK>', '<UNK>'): 7104, ('<UNK>', 'a'): 2937, ('a', '<UNK>'): 2864, ('<UNK>', 'you'): 1288, ('you', '<UNK>'): 1293, ('<UNK>', 'in'): 1804, ('in', '<UNK>'): 1730, ('in', 'a'): 1620, ('a', 'a'): 2522, ('<UNK>', 'the'): 5344}

UNK Laplace bigram probabilities (subset):
{('<UNK>', '<UNK>'): 0.0538690160280225, ('<UNK>', 'a'): 0.02

## Linear Interpolation

In [11]:
def linear_interpolation(sentence, lm_type):
    # Set the lambda values for each n-gram model
    lambda_3 = 0.6
    lambda_2 = 0.3
    lambda_1 = 0.1
    
    # Tokenize the sentence
    sentence_tokens = sentence.split()
    
    # Calculate the probabilities of each n-gram model for the sentence
    if lm_type == "vanilla":
        unigram_probs = [vanilla_unigram_probs[token] for token in sentence_tokens]
        bigram_probs = [vanilla_bigram_probs.get((sentence_tokens[i-1], sentence_tokens[i]), 0) for i in range(1, len(sentence_tokens))]
        trigram_probs = [vanilla_trigram_probs.get((sentence_tokens[i-2], sentence_tokens[i-1], sentence_tokens[i]), 0) for i in range(2, len(sentence_tokens))]
    elif lm_type == "laplace":
        unigram_probs = [(laplace_unigram_freq.get(token, 0) + 1) / (laplace_total_unigrams + len(laplace_unigram_freq)) for token in sentence_tokens]
        bigram_probs = [(laplace_bigram_freq.get((sentence_tokens[i-1], sentence_tokens[i]), 0) + 1) / (laplace_unigram_freq.get(sentence_tokens[i-1], 0) + len(laplace_unigram_freq)) for i in range(1, len(sentence_tokens))]
        trigram_probs = [(laplace_trigram_freq.get((sentence_tokens[i-2], sentence_tokens[i-1], sentence_tokens[i]), 0) + 1) / (laplace_bigram_freq.get((sentence_tokens[i-2], sentence_tokens[i-1]), 0) + len(laplace_unigram_freq)) for i in range(2, len(sentence_tokens))]
    elif lm_type == "unk":
        # Replace infrequent words with <UNK>
        unk_sentence_tokens = ['<UNK>' if word_freq[token] <= 2 else token for token in sentence_tokens]
        # Calculate the probabilities for the UNK Laplace model
        unk_laplace_unigram_probs = {word: (count + 1) / (unk_laplace_total_unigrams + len(unk_laplace_unigram_freq)) for word, count in unk_laplace_unigram_freq.items()}
        unk_laplace_bigram_probs = {(word1, word2): (count + 1) / (unk_laplace_unigram_freq.get(word1, 0) + len(unk_laplace_unigram_freq)) for (word1, word2), count in unk_laplace_bigram_freq.items()}
        unk_laplace_trigram_probs = {(word1, word2, word3): (count + 1) / (unk_laplace_bigram_freq.get((word1, word2), 0) + len(unk_laplace_unigram_freq)) for (word1, word2, word3), count in unk_laplace_trigram_freq.items()}
        unk_laplace_trigram_probs = {(word1, word2, word3): (count + 1) / (unk_laplace_bigram_freq.get((word1, word2), 0) + len(unk_laplace_unigram_freq)) for (word1, word2, word3), count in unk_laplace_trigram_freq.items()}
        # Replace tokens in sentence with <UNK> where appropriate
        unigram_probs = [unk_laplace_unigram_probs.get(token, 1 / (unk_laplace_total_unigrams + len(unk_laplace_unigram_freq))) for token in unk_sentence_tokens]
        bigram_probs = [(unk_laplace_bigram_freq.get((unk_sentence_tokens[i-1], unk_sentence_tokens[i]), 0) + 1) / (unk_laplace_unigram_freq.get(unk_sentence_tokens[i-1], 0) + len(unk_laplace_unigram_freq)) for i in range(1, len(unk_sentence_tokens))]
        trigram_probs = [(unk_laplace_trigram_probs.get((unk_sentence_tokens[i-2], unk_sentence_tokens[i-1], unk_sentence_tokens[i]), 0) + 1) / (unk_laplace_bigram_freq.get((unk_sentence_tokens[i-2], unk_sentence_tokens[i-1]), 0) + len(unk_laplace_unigram_freq)) for i in range(2, len(unk_sentence_tokens))]
        
    # Calculate the probability of the sentence using linear interpolation
    sentence_prob = 1
    for i in range(len(sentence_tokens)):
        if i == 0:
            sentence_prob *= unigram_probs[i]**lambda_1
        elif i == 1:
            sentence_prob *= (lambda_2*bigram_probs[i-1] + lambda_1*unigram_probs[i])**lambda_1
        else:
            sentence_prob *= (lambda_3*trigram_probs[i-2] + lambda_2*bigram_probs[i-1] + lambda_1*unigram_probs[i])**lambda_1

    return sentence_prob

# Define some test sentences
sentences = ["this is a test sentence"]

# Test the vanilla language model
for sentence in sentences:
    prob = linear_interpolation(sentence, "vanilla")
    print(f"\nVanilla LM: Probability of '{sentence}': {prob:.6f}")

# Test the laplace language model
for sentence in sentences:
    prob = linear_interpolation(sentence, "laplace")
    print(f"Laplace LM: Probability of '{sentence}': {prob:.6f}")

# Test the unk language model
for sentence in sentences:
    prob = linear_interpolation(sentence, "unk")
    print(f"UNK LM: Probability of '{sentence}': {prob:.6f}")


Vanilla LM: Probability of 'this is a test sentence': 0.035839
Laplace LM: Probability of 'this is a test sentence': 0.024603
UNK LM: Probability of 'this is a test sentence': 0.027749


## Evaluation & Perplexity

In [12]:
# This function calculates the perplexity of a given test set using the specified n-gram language model probabilities
def calculate_perplexity(test_set, lm_probs, ngram_type):
    # Get the length of the test set
    N = len(test_set)
    # Initialize the sum of the log probabilities to 0
    log_prob_sum = 0

    # Iterate over the test set and calculate the log probability of each n-gram in the test set
    for i in range(len(test_set)):
        if ngram_type == "unigram":
            # Calculate the log probability of the unigram
            log_prob_sum += math.log(lm_probs.get(test_set[i], 1))
        elif ngram_type == "bigram" and i > 0:
            # Calculate the log probability of the bigram
            log_prob_sum += math.log(lm_probs.get((test_set[i-1], test_set[i]), 1))
        elif ngram_type == "trigram" and i > 1:
            # Calculate the log probability of the trigram
            log_prob_sum += math.log(lm_probs.get((test_set[i-2], test_set[i-1], test_set[i]), 1))

    # Calculate the entropy and perplexity of the test set using the log probabilities
    entropy = -(log_prob_sum / N)
    perplexity = math.pow(2, entropy)

    # Return the perplexity
    return perplexity

# Calculate perplexities for each model and n-gram
vanilla_unigram_perplexity = calculate_perplexity(test_set, vanilla_unigram_probs, "unigram")
vanilla_bigram_perplexity = calculate_perplexity(test_set, vanilla_bigram_probs, "bigram")
vanilla_trigram_perplexity = calculate_perplexity(test_set, vanilla_trigram_probs, "trigram")

laplace_unigram_perplexity = calculate_perplexity(test_set, laplace_unigram_probs, "unigram")
laplace_bigram_perplexity = calculate_perplexity(test_set, laplace_bigram_probs, "bigram")
laplace_trigram_perplexity = calculate_perplexity(test_set, laplace_trigram_probs, "trigram")

unk_unigram_perplexity = calculate_perplexity(test_set, unk_laplace_unigram_probs, "unigram")
unk_bigram_perplexity = calculate_perplexity(test_set, unk_laplace_bigram_probs, "bigram")
unk_trigram_perplexity = calculate_perplexity(test_set, unk_laplace_trigram_probs, "trigram")

# Print the results in a table format
print("\n{:<15} {:<15} {:<15} {:<15}".format("Model", "Unigram", "Bigram", "Trigram"))
print("-" * 60)
print("{:<15} {:<15.2f} {:<15.2f} {:<15.2f}".format("Vanilla", vanilla_unigram_perplexity, vanilla_bigram_perplexity, vanilla_trigram_perplexity))
print("{:<15} {:<15.2f} {:<15.2f} {:<15.2f}".format("Laplace", laplace_unigram_perplexity, laplace_bigram_perplexity, laplace_trigram_perplexity))
print("{:<15} {:<15.2f} {:<15.2f} {:<15.2f}".format("UNK", unk_unigram_perplexity, unk_bigram_perplexity, unk_trigram_perplexity))


Model           Unigram         Bigram          Trigram        
------------------------------------------------------------
Vanilla         120.41          13.59           1.53           
Laplace         120.94          36.10           2.54           
UNK             110.36          27.62           2.35           


## Genetarion

In [13]:
# Define a function to generate a sentence using a language model and a given phrase
def generate_sentence(model, phrase):
    # Print a message indicating which model is being used
    print(f"\nGenerating {model} model...")
    print("-" * 127)
    # Loop through each n-gram type and generate a sentence for each type
    for ngram_type in ["unigram", "bigram", "trigram"]:
        # Generate a sentence using the selected model, phrase, and n-gram type
        generated_sentence = generate_ngram_sentence(model, phrase, ngram_type)
        # Print the generated sentence along with its n-gram type
        print(f"\n{ngram_type.capitalize()}: {generated_sentence}\n")
        print("-" * 127)

# Define a function to generate a sentence using a specific n-gram type and language model
def generate_ngram_sentence(model, phrase, ngram_type):
    # Define start and end tokens for the sentence
    start_token = '<s>'
    end_token = '</s>'
    # Set a maximum length for the sentence
    max_length = 20
    # Initialize the sentence with the start token and the words in the input phrase
    sentence = [start_token] + phrase.split()
    # Initialize the model probabilities to None
    model_probs = None
    
    # Convert the model name to lowercase for easier comparison
    model = model.lower()
    
    # Determine which set of probabilities to use based on the selected model
    if model == 'vanilla':
        model_probs = [vanilla_unigram_probs, vanilla_bigram_probs, vanilla_trigram_probs]
    elif model == 'laplace':
        model_probs = [laplace_unigram_probs, laplace_bigram_probs, laplace_trigram_probs]
    elif model == 'unk':
        model_probs = [unk_laplace_unigram_probs, unk_laplace_bigram_probs, unk_laplace_trigram_probs]
    else:
        # If an invalid model was selected, print an error message and return None
        print('Invalid model selection')
        return

    # Generate the sentence by adding words one at a time until the end token is reached or the maximum length is reached
    while sentence[-1] != end_token and len(sentence) < max_length:
        # Determine the next word to add based on the selected n-gram type
        if ngram_type == "unigram":
            next_word = generate_word(model_probs[0])
        elif ngram_type == "bigram":
            next_word = generate_word(model_probs[1], sentence[-1:])
        elif ngram_type == "trigram":
            if len(sentence) < 2:
                next_word = generate_word(model_probs[1], sentence[-1:])
            else:
                next_word = generate_word(model_probs[2], sentence[-2:])
        # Add the next word to the sentence
        sentence.append(next_word)

    # Return the generated sentence, excluding the start token
    return ' '.join(sentence[1:])

# Define a function to generate a word given a probability distribution
def generate_word(model_prob, context=None):
    # If no context is provided, use the full vocabulary and corresponding probabilities
    if context is None:
        words, probs = zip(*[(word, prob) for word, prob in model_prob.items()])
    # If a context is provided, use only the vocabulary and probabilities corresponding to that context
    else:
        words, probs = zip(*[(word, model_prob.get(tuple(context + [word]), 0)) for word in vocab])
        
    # Cast the probabilities to a numpy array of float64 data type    
    probs = np.array(probs, dtype=np.float64)
    epsilon = 1e-10
    probs = np.add(probs, epsilon)  # Add a small constant to the probabilities
    probs /= probs.sum()
    next_word = np.random.choice(words, p=probs)
    return next_word

# Ask the user for the model and phrase input
model_input = input('\nWhich language model would you like to use? (Vanilla, Laplace, UNK): ').lower()  # Convert to lowercase
phrase_input = input('Please enter a phrase: ')

# Generate the sentence using the selected model and input phrase
generate_sentence(model_input, phrase_input)


Which language model would you like to use? (Vanilla, Laplace, UNK): UNK
Please enter a phrase: lets go to the

Generating unk model...
-------------------------------------------------------------------------------------------------------------------------------

Unigram: lets go to the at from his will it to report analyse wa mamur the many uncovered this on

-------------------------------------------------------------------------------------------------------------------------------

Bigram: lets go to the to cargo wave want potential the we the century old represent against seven his aside

-------------------------------------------------------------------------------------------------------------------------------

Trigram: lets go to the have broader health for much control too party profit not never and they daily said

-------------------------------------------------------------------------------------------------------------------------------


### Test output for Generation function

In [16]:
# Test the sentence generation function with different models and phrases
models = ['Vanilla', 'Laplace', 'UNK']
phrase = ['this is a']

for model in models:
    for sentence in phrase:
        generate_sentence(model, sentence)
        print("-" * 123)


Generating Vanilla model...
-------------------------------------------------------------------------------------------------------------------------------

Unigram: this is a identifying but be meeting give that up confident month in standing time about after research problem

-------------------------------------------------------------------------------------------------------------------------------

Bigram: this is a that a mm broken petal i an feed their community root to talk or of claire

-------------------------------------------------------------------------------------------------------------------------------

Trigram: this is a have rome what pimp fact desperate be the let it go must what other to a

-------------------------------------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------------------------------

Generating Laplace mod

## Sen_Probability

In [15]:
def sen_probability(sentence, model_name, ngram_type):
    # Tokenize sentence into words
    sentence_tokens = sentence.lower().split()

    # Initialize dictionaries to store sentence probabilities
    sentence_probs = {
        "Vanilla": {"unigram": 1, "bigram": None, "trigram": None},
        "Laplace": {"unigram": 1, "bigram": None, "trigram": None},
        "UNK": {"unigram": 1, "bigram": None, "trigram": None}
    }

    # Calculate probabilities for each model and n-gram type
    for i in range(len(sentence_tokens)):
        # Iterate through each model and corresponding probabilities
        for model_name, model_probs in [("Vanilla", (vanilla_unigram_probs, vanilla_bigram_probs, vanilla_trigram_probs)),
                                        ("Laplace", (laplace_unigram_probs, laplace_bigram_probs, laplace_trigram_probs)),
                                        ("UNK", (unk_laplace_unigram_probs, unk_laplace_bigram_probs, unk_laplace_trigram_probs))]:
            # For the first word, calculate the unigram probability and multiply it by the running total
            if i == 0:
                sentence_probs[model_name]["unigram"] *= model_probs[0].get(sentence_tokens[i], 1)
            # For the second word, calculate the bigram probability and multiply it by the running total
            elif i == 1:
                if sentence_probs[model_name]["bigram"] is None:
                    sentence_probs[model_name]["bigram"] = 1
                sentence_probs[model_name]["bigram"] *= model_probs[1].get((sentence_tokens[i-1], sentence_tokens[i]), 1)
            # For all subsequent words, calculate the trigram probability and multiply it by the running total
            else:
                if sentence_probs[model_name]["trigram"] is None:
                    sentence_probs[model_name]["trigram"] = 1
                sentence_probs[model_name]["trigram"] *= model_probs[2].get((sentence_tokens[i-2], sentence_tokens[i-1], sentence_tokens[i]), 1)

    # Print probabilities for the specified model and n-gram type
    print(f"\nSentence: {sentence}")
    print(f"{model_name} {ngram_type} sentence probability:", sentence_probs[model_name][ngram_type])

# Test the sen_probability function with a sample sentence, model, and n-gram type
sentence = "lets go to the bank"
model_name = "Vanilla"
ngram_type = "bigram"

prob = sen_probability(sentence, model_name, ngram_type)


Sentence: lets go to the bank
UNK bigram sentence probability: 1
